In [1]:
import pandas as pd
import numpy as np
import time
import plotly.express as px
from pmdarima.arima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import acovf,acf,pacf
from sklearn.metrics import mean_squared_error

In [2]:
store_df = pd.read_csv('data/store.csv')
test_df = pd.read_csv('data/test.csv')
train_df = pd.read_csv('data/train.csv')
sample_submission_df = pd.read_csv('data/sample_submission.csv')

/scratch/7789725.1.huggins-pub/ipykernel_1584930/3598239639.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('data/train.csv')


In [3]:
test_df

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0
...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0
41084,41085,1112,6,2015-08-01,1.0,0,0,0
41085,41086,1113,6,2015-08-01,1.0,0,0,0
41086,41087,1114,6,2015-08-01,1.0,0,0,0


## Data preprocessing

#### Checking any problems with the data

In [4]:
train_df.StateHoliday.unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

The value 0 occurs in two data types

In [5]:
train_df.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [6]:
def one_hot_encode(df, column_name):
    one_hot_df = pd.get_dummies(df[column_name], prefix=column_name+"_", drop_first=False, dtype=int)
    new_cols = df.columns.drop(column_name)
    new_df = pd.concat([df[new_cols], one_hot_df], axis=1)
    return new_df

def preprocess_data(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df.loc[df['StateHoliday'] == 0, 'StateHoliday'] = '0'
    df = one_hot_encode(df, 'StateHoliday')
    df = df.sort_values(by=['Store', 'Date']).reset_index(drop=True)
    # df = df.iloc[::-1].reset_index(drop=True)
    
    return df
    

In [7]:
train_df = preprocess_data(train_df)
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,StateHoliday__0,StateHoliday__a,StateHoliday__b,StateHoliday__c
0,1,2,2013-01-01,0,0,0,0,1,0,1,0,0
1,1,3,2013-01-02,5530,668,1,0,1,1,0,0,0
2,1,4,2013-01-03,4327,578,1,0,1,1,0,0,0
3,1,5,2013-01-04,4486,619,1,0,1,1,0,0,0
4,1,6,2013-01-05,4997,635,1,0,1,1,0,0,0


In [8]:
test_df.shape

(41088, 8)

In [9]:
orig_test_df = test_df.copy()
test_df = preprocess_data(test_df)
test_df["StateHoliday__b"] = len(test_df) * [0]
test_df["StateHoliday__c"] = len(test_df) * [0]
test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,SchoolHoliday,StateHoliday__0,StateHoliday__a,StateHoliday__b,StateHoliday__c
0,40233,1,6,2015-08-01,1.0,0,1,1,0,0,0
1,39377,1,7,2015-08-02,0.0,0,1,1,0,0,0
2,38521,1,1,2015-08-03,1.0,1,1,1,0,0,0
3,37665,1,2,2015-08-04,1.0,1,1,1,0,0,0
4,36809,1,3,2015-08-05,1.0,1,1,1,0,0,0


In [10]:
orig_test_df = orig_test_df.sort_values(by=['Store', 'Date']).reset_index(drop=True)
orig_test_df["Sales"] = len(orig_test_df) * [0]
orig_test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Sales
0,40233,1,6,2015-08-01,1.0,0,0,1,0
1,39377,1,7,2015-08-02,0.0,0,0,1,0
2,38521,1,1,2015-08-03,1.0,1,0,1,0
3,37665,1,2,2015-08-04,1.0,1,0,1,0
4,36809,1,3,2015-08-05,1.0,1,0,1,0


In [11]:
test_df["Store"].unique()[:10]

array([ 1,  3,  7,  8,  9, 10, 11, 12, 13, 14])

In [ ]:
# train_df["Store"].unique()
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()
for store_id in test_df["Store"].unique():
   print("Training store: ", store_id)
   
   store_train_df = train_df[train_df["Store"] == store_id]
   store_test_df = test_df[test_df["Store"] == store_id]

   exog_cols = ['DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'StateHoliday__0', 'StateHoliday__a',
      'StateHoliday__b', 'StateHoliday__c']
   # exog = store_train_df[exog_cols].to_numpy()

   model = SARIMAX(store_train_df['Sales'], exog=store_train_df[exog_cols], order=(1, 1, 2), seasonal_order=(2, 0, 0, 7))
   model_fit = model.fit(disp=False)

   start = len(store_train_df)
   end = start + len(store_test_df) - 1

   # print("From: ", start, " To: ", end)

   forecast = model_fit.predict(start=start, end=end, exog=store_test_df[exog_cols])
   orig_test_df.loc[orig_test_df["Store"] == store_id, "Sales"] = forecast.values
    
   # print("\n")

print("Time taken: ", time.time() - start_time)

Training store:  1
Training store:  3
Training store:  7
Training store:  8
Training store:  9
Training store:  10
Training store:  11
Training store:  12
Training store:  13
Training store:  14
Training store:  15
Training store:  16
Training store:  19
Training store:  20
Training store:  21
Training store:  22
Training store:  23
Training store:  24
Training store:  25
Training store:  27
Training store:  29
Training store:  30
Training store:  31


In [ ]:
orig_test_df

In [ ]:
orig_test_df.to_csv("data/test_df.csv", index=False)

## Conclusion

The SARIMAX model with exogenous variables is performing significantly better. The correlation plots show that sales data is related to the Customer, Close, and Open columns, so it makes sense that the model's performance has improved after including this data.
